In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import mysql.connector
import re

In [28]:
column_names = ['title','url','date']
new_df = pd.DataFrame(columns = column_names)

In [29]:
idx = 0

In [30]:
chromedriver_path = "C:/Users/kkksk/.wdm/drivers/chromedriver/win64/125.0.6422.78/chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

In [31]:
# cur_month = 7
cur_year = 2007
target_year = 2007

while cur_year <= target_year:
    
    url = f"https://www.google.com/search?q=oil&tbs=cdr:1,cd_min:1/1/{cur_year},cd_max:12/31/{cur_year}&tbm=nws"
    driver.get(url)
    
    time.sleep(30)
    
    for _ in range(30):
        news_results = driver.find_elements(By.CSS_SELECTOR, 'div#rso > div >div>div>div')
        for news_div in news_results:
            try:
                news_link = news_div.find_element(By.TAG_NAME, 'a').get_attribute('href')
                news_text_elements = news_div.find_elements(By.CSS_SELECTOR, 'a>div>div>div')
                news_item = [elem.text for elem in news_text_elements]
                if len(news_item) >= 5:
                    Link = news_link
                    Title = news_item[2]
                    Date = news_item[4]
                
                new_df.loc[idx] = [Title, Link, Date]
                idx += 1
            except Exception as e:
                print(e)
        
        try:
            next_button = driver.find_element(By.ID, 'pnnext')
            next_button.click()
            time.sleep(1)
        except NoSuchElementException:
            break
    # if cur_month == 7:  
    #     cur_month = 1
    cur_year += 1
    # else: 
    #     cur_month += 6


In [33]:
new_df

,title,url,date
0,2nd Oil Shock?,https://www.koreatimes.co.kr/www/opinion/2024/...,2007. 10. 21.
1,2nd Oil Shock?,https://www.koreatimes.co.kr/www/opinion/2024/...,2007. 10. 21.
2,"Oil Nationalism, the Oil Industry and Energy S...",https://www.realinstitutoelcano.org/en/analyse...,2007. 10. 25.
3,What are the possible causes and consequences ...,https://www.frbsf.org/research-and-insights/pu...,2007. 11. 1.
4,Cytotoxicity of silicone oil on cultivated hum...,https://www.nature.com/articles/6702962,2007. 8. 24.
...,...,...,...
295,"Tapped Out, but Hopeful: A Break in Texas's Oi...",https://www.nytimes.com/2007/11/02/business/02...,2007. 11. 2.
296,Pasha Bulker Swung Out but Leaking Oil,https://gcaptain.com/pasha-bulker-swung-out-bu...,2007. 7. 1.
297,Standing on oil – Twin Cities,https://www.twincities.com/2007/12/08/standing...,2007. 12. 8.
298,Brent oil surges closer to record,https://www.rte.ie/news/business/2007/0716/912...,2007. 7. 16.


In [34]:
new_df.to_parquet('news_oil_07.parquet', engine = 'pyarrow', compression = 'gzip', index = False)